In [1]:
import numpy as np
import pandas as pd 
from keras.utils import np_utils
import os
import keras
import gensim
from random import randint
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
%matplotlib inline
import matplotlib.pyplot as plt
from numpy import array
from keras.utils import to_categorical
from numpy import argmax
from keras import Sequential
from keras.layers import LSTM,Input,Lambda,concatenate, Dense,Dropout,GRU,BatchNormalization
from keras.utils import to_categorical
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text as keras_text, sequence as keras_seq
import re
import keras.backend as K
from keras.models import Model
from keras.optimizers import Adadelta
from time import time
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
import tensorflow as tf
from keras import optimizers
from sklearn.model_selection import train_test_split,StratifiedKFold

Using TensorFlow backend.


In [2]:
def ReadData():
    train1 = pd.read_csv("../input/homedepot/train.csv",encoding ='latin1')
    test1 = pd.read_csv("../input/home-depot-product-search-relevance/test.csv",encoding ='latin1')
    return train1,test1
def ConcatDescAndTitle(df):
    train1 = df.merge(desc,on='product_uid',how='left')
    #train1['TitleAndDesc'] = train1[['product_title', 'product_description']].apply(lambda x: ''.join(x), axis=1)
    train1 = train1.drop(columns=['product_description','id'])
    #train1['product_title'] = train1['product_title'].str.replace('[^a-zA-Z0-9\s]','$')
    #train1['search_term'] = train1['search_term'].str.replace('[^a-zA-Z0-9\s]','$')
    train1['product_title'] = train1['product_title'].str.replace('\\xa0', '')
    train1['search_term'] = train1['search_term'].str.replace('\\xa0', '')
    return train1
def getBrandAndMatirial():
    for i in range(test.shape[0]):
        row = test.loc[i]
        #foo = df.ix[(df['column1']==value) | (df['columns2'] == 'b') | (df['column3'] == 'c')]
        row2 = attributes[attributes["product_uid"]==row["product_uid"]]
        brand = row2[row2["name"]==str("MFG Brand Name")]["value"]
        Material = row2[row2["name"]==str("Material")]["value"]
        if ((brand.values.any())):
            if ((Material.values.any())):
                train.loc[i,"product_title"] += " " + str(brand.values[0]) + " " + str(Material.values[0])
    return train
def RemoveNotCommonRelevances():
    train2 = train
    train2 = train2[train2.relevance != 1.25]
    train2 = train2[train2.relevance != 1.5]
    train2 = train2[train2.relevance != 2.5]
    train2 = train2[train2.relevance != 1.75]
    train2 = train2[train2.relevance != 2.5]
    train2 = train2[train2.relevance != 2.25]
    train2 = train2[train2.relevance != 2.75]
    return train2.reset_index(drop=True)
def AddMoreIRelevance(train):
    size = len(chars)
    for i in range(40000):
        if (i%5000==0):
            print(i)
        title = ""
        term = ""
        for j in range(67):
            title += chars[randint(0, size-1)]
            term += chars[randint(0, size-1)]
        rander = randint(0, 1)
        rel = 0
        if (rander==0):
            rel = 1.0
        if (rander==1):
             rel = 1.33
        train = train.append({'product_title':title,'search_term':term,"relevance":rel }, ignore_index=True)
    return train

In [17]:
chars = "abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
lenthofchar = len(chars)
train,test = ReadData()
train = RemoveNotCommonRelevances()

,id,product_uid,product_title,search_term
0,1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket
1,4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets
2,5,100001,Simpson Strong-Tie 12-Gauge Angle,simpson sku able
3,6,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong ties
4,7,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong tie hcc668


In [5]:
def CreateDic():
    CharToIndex2 = {}
    counter=0
    for key in model.wv.vocab.keys():
        CharToIndex2[key] = counter
        counter += 1
    return CharToIndex2
def CreateOneHot(index):
    arr = np.zeros(lenthofchar)
    if (index!=-1):
        arr[index] = 1
    return arr
def CreateCategorial(y):
    UniquToPred2 = {}
    PredToUnique2 = {}
    u = np.unique(y)
    for i in range(len(u)):
        UniquToPred2[i] = u[i]
        PredToUnique2[u[i]] = i
    return UniquToPred2,PredToUnique2
def CreateOneHotLength(index,length):
    arr = np.zeros(length)
    arr[index] = 1
    return arr
def CreateY(df):
    y = np.zeros((df.shape[0],7))
    for i in range(df.shape[0]):
            y[i] = CreateOneHotLength(PredToUnique[df[i]],7)
    return y

****Train Test Split with suffle****

In [6]:
def spliting(x,trainsize):
    X_train, X_test = train_test_split(x, test_size=1-trainsize, shuffle=True)
    X_train = X_train.reset_index()
    X_train = X_train.drop(columns=['index'])
    X_test = X_test.reset_index()
    X_test = X_test.drop(columns=['index'])
    return X_train, X_test,X_train.relevance,X_test.relevance
def LeftRightAndYTrain(df,y,index):
    left = df[0][index:]
    right = df[1][index:]
    y = y[index]
    y = CreateY(y)
    return left,right,y
def LeftRightAndYTest(df,y,index):
    left = df[0][index]
    right = df[1][index]
    y = y[index]
    return left,right,y
def ReplaceCharsWithOneHotMaxLen(temp):
    avglen = 0
    for i in range(temp.shape[0]):
        lis =(list(temp.loc[i]["product_title"]))
        avglen += len(lis)
    avglen = int(math.floor(avglen/temp.shape[0]))
    avglen = 50
    final_df = np.empty([2,temp.shape[0],avglen,lenthofchar])
    for i in range(final_df.shape[1]):
        lis =(list(temp.loc[i]["product_title"]))
        lis2 =(list(temp.loc[i]["search_term"]))
        for j in range(final_df.shape[2]):
            if (j<len(lis)):
                final_df[0,i,j]=CreateOneHot(chars.find(lis[j]))
            else:
                final_df[0,i,j]= CreateOneHot(-1)
            if (j<len(lis2)):
                final_df[1,i,j]= CreateOneHot(chars.find(lis2[j]))
            else:
                final_df[1,i,j]=CreateOneHot(-1)
    return final_df
def ConveteBackToPred():
    newPred = np.zeros(preds2.shape[0],)
    for i in range(preds2.shape[0]):
        maxim=float(0)
        indx=0
        for j in range(preds2[i].size):
            if (preds[i][j]>maxim):
                indx = j
                maxim = preds2[i][j]
        newPred[i] = UniquToPred[indx] 
    return newPred      

In [7]:
train1, test1,y_train,y_test = spliting(train,0.8)
UniquToPred,PredToUnique = CreateCategorial(train["relevance"].astype(str))
y_train = y_train.values
y_train = y_train.astype(str)
y_train = CreateY(y_train)
train1 = ReplaceCharsWithOneHotMaxLen(train1)
test1 = ReplaceCharsWithOneHotMaxLen(test1)
test = ReplaceCharsWithOneHotMaxLen(test)

**trying the Kfold**

In [8]:
def cossim(left, right):
    ans = (K.sum(left*right))/(K.sqrt(K.sum(K.pow(left,2)))*K.sqrt(K.sum(K.pow(right,2))))
    ans = tf.nn.relu(ans)
    return ans*2 +1
def custum_mean_squared_error(y_true, y_pred):
    return K.mean(K.square(((y_pred - y_true)))+(y_pred - y_true)*margin, axis=-1)

In [10]:
def getModel():
    left_input = Input(shape=(X_train_left_kfold.shape[1], X_train_left_kfold.shape[2]))
    right_input = Input(shape=(X_train_right_kfold.shape[1], X_train_right_kfold.shape[2]))
    shared_lstm = LSTM(n_hidden)
    left_out = shared_lstm(left_input)
    right_out = shared_lstm(right_input)
    left_out_norm = BatchNormalization()(left_out)
    right_out_norm = BatchNormalization()(right_out)
    merged_vector = concatenate([left_out_norm, right_out_norm], axis=-1)
    #malstm_distance = Lambda(function=lambda x: cossim(x[0], x[1]),output_shape=lambda x: (1, 1))([left_out, right_out])
    predictions2 = Dense(20,activation="sigmoid")(merged_vector)
    predictions4 = Dense(7,activation="softmax")(predictions2)
    # Pack it all up into a model
    malstm = Model([left_input, right_input], [predictions4])

    #optimizer = Adadelta(clipnorm=1.25)
    #malstm.compile(loss='mse', optimizer="adam", metrics=['accuracy'])
    #sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)

    malstm.compile(loss='mse', optimizer="adam")
    malstm.summary()
    return malstm

**THE LSTM NetWork**

In [11]:
n_hidden = 100
batch_size = 526
total_rmse = 0
total_mae = 0
n_epoch = 3
X_train_left_kfold = train1[0]
X_train_right_kfold = train1[1]
X_test_left_kfold = test1[0]
X_test_right_kfold = test1[1]
malstm1 = getModel()
training_start_time = time()
malstm_trained = malstm1.fit([X_train_left_kfold,X_train_right_kfold],y_train, batch_size=batch_size, nb_epoch=n_epoch,validation_split=0.05)
#preds = malstm1.predict([X_test_left_kfold,X_test_right_kfold], batch_size=batch_size)
#pred = ConveteBackToPred()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50, 63)       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 50, 63)       0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 100)          65600       input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 100)          400         lstm_1[0][0]                     
__________

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if sys.path[0] == '':


Train on 56245 samples, validate on 2961 samples
Epoch 1/3
56245/56245 [==============================] - 60s 1ms/step - loss: 0.1157 - val_loss: 0.1174
Epoch 2/3
56245/56245 [==============================] - 58s 1ms/step - loss: 0.1143 - val_loss: 0.1146
Epoch 3/3
56245/56245 [==============================] - 57s 1ms/step - loss: 0.1141 - val_loss: 0.1143


In [ ]:
preds2 = malstm1.predict([test[0],test[1]], batch_size=batch_size)
pred2 = ConveteBackToPred()
train,test = ReadData()
sample_sub = pd.DataFrame()
sample_sub['id'] = test['id']
sample_sub['relevance'] = pred2
sample_sub.to_csv('charfinal.csv', index=False)